In [ ]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM(), simulation=False)
        node.start()
        nodes.append(node)

    print("a")
        
    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port,full=True)
        time.sleep(.1)
        print("a")
        

    time.sleep(3)     
        
    # Check if they are connected
    for node in nodes:
        assert len(node.neightbors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1)  
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:49545
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:33419
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:34551
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:45773
a


In [7]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port,full=True)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightbors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(7)
    nodes[1].stop()

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:58421
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:59085
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:36879
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:54231
INFO:root:127.0.0.1:59085 Connected to 127.0.0.1:58421
INFO:root:127.0.0.1:58421 Conexión aceptada con 127.0.0.1:59085
INFO:root:127.0.0.1:36879 Connected to 127.0.0.1:59085
INFO:root:127.0.0.1:59085 Conexión aceptada con 127.0.0.1:36879
INFO:root:127.0.0.1:58421 Connected to 127.0.0.1:36879
INFO:root:127.0.0.1:36879 Conexión aceptada con 127.0.0.1:58421
INFO:root:127.0.0.1:54231 Connected to 127.0.0.1:36879
INFO:root:127.0.0.1:36879 Conexión aceptada con 127.0.0.1:54231
INFO:root:127.0.0.1:58421 Connected to 127.0.0.1:54231
INFO:root:127.0.0.1:59085 Connected to 127.0.0.1:54231
INFO:root:127.0.0.1:54231 Conexión aceptada con 127.0.0.1:58421
INFO:root:127.0.0.1:54231 Conexión aceptad

INFO:root:(127.0.0.1:54231) Round 1 of 2 finished.
INFO:root:(127.0.0.1:58421) Self Vote: [('127.0.0.1:58421', 76), ('127.0.0.1:54231', 476), ('127.0.0.1:36879', 156)]
INFO:root:(127.0.0.1:36879) Round 1 of 2 finished.
INFO:root:(127.0.0.1:54231) Sending train set vote.
INFO:root:(127.0.0.1:58421) Waiting other node votes.
INFO:root:(127.0.0.1:36879) Sending train set vote.
INFO:root:(127.0.0.1:54231) Self Vote: [('127.0.0.1:36879', 180), ('127.0.0.1:59085', 240), ('127.0.0.1:58421', 193)]
INFO:root:(127.0.0.1:36879) Self Vote: [('127.0.0.1:58421', 23), ('127.0.0.1:59085', 319), ('127.0.0.1:54231', 71)]
INFO:root:(127.0.0.1:54231) Waiting other node votes.
INFO:root:(127.0.0.1:36879) Waiting other node votes.
INFO:root:(127.0.0.1:58421) Timeout for vote agregation. Missing votes from {'127.0.0.1:59085'}
FUNCTION NOT IMPLEMETED
INFO:root:(127.0.0.1:36879) Timeout for vote agregation. Missing votes from {'127.0.0.1:59085'}
FUNCTION NOT IMPLEMETED
INFO:root:(127.0.0.1:54231) Timeout for v

In [1]:
import pytest
from p2pfl.base_node import BaseNode
from p2pfl.communication_protocol import CommunicationProtocol
import time
import pytest
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
from p2pfl.settings import Settings


def four_nodes():
    n1 = Node(MLP(),MnistFederatedDM())
    n2 = Node(MLP(),MnistFederatedDM())
    n3 = Node(MLP(),MnistFederatedDM())
    n4 = Node(MLP(),MnistFederatedDM())
    n1.start()
    n2.start()
    n3.start()
    n4.start()

    return n1,n2,n3,n4

def test_gossip_heartbeat():

    n1.connect_to(n2.host,n2.port, full=False)
    n2.connect_to(n3.host,n3.port, full=False)
    n3.connect_to(n4.host,n4.port, full=False)

    time.sleep(2) # Wait for asincronity    

    assert len(n1.get_network_nodes()) == len(n2.get_network_nodes()) == len(n3.get_network_nodes()) == len(n4.get_network_nodes()) == 3

    
    n1.set_start_learning(rounds=2,epochs=0)    

    while all([n1.round is not None for n in [n1,n2,n3,n4]]):
        time.sleep(0.1)

    for n in [n1,n2,n3,n4]:
        n.stop()
        time.sleep(0.1)

n1, n2, n3, n4 = four_nodes()
test_gossip_heartbeat()

Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:37107
INFO:root:Nodo listening at 127.0.0.1:37339
INFO:root:Nodo listening at 127.0.0.1:40133
INFO:root:Nodo listening at 127.0.0.1:55395
INFO:root:127.0.0.1:37107 Connected to 127.0.0.1:37339
INFO:root:127.0.0.1:37339 Conexión aceptada con 127.0.0.1:37107
INFO:root:127.0.0.1:37339 Connected to 127.0.0.1:40133
INFO:root:127.0.0.1:40133 Conexión aceptada con 127.0.0.1:37339
INFO:root:127.0.0.1:40133 Connected to 127.0.0.1:55395
INFO:root:127.0.0.1:55395 Conexión aceptada con 127.0.0.1:40133
INFO:root:(127.0.0.1:37107) Broadcasting start learning...
INFO:root:(127.0.0.1:37339) Waiting initialization.
INFO:root:(127.0.0.1:37107) Waiting initialization.
INFO:root:(127.0.0.1:37107) Gossiping model initialization.
INFO:root:(127.0.0.1:37107) Gossiping model to 1 train set nodes.
INFO:root:(127.0.0.1:40133) Waiting initializa

INFO:root:(127.0.0.1:37339) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:37107) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (1/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:40133) Model added (2/4) from ['127.0.0.1:37339']
INFO:root:(127.0.0.1:40133) Model added (3/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Model added (3/4) from ['127.0.0.1:37107']
INFO:root:(127.0.0.1:37339) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:55395) Model added (4/4) from ['127.0.0.1:37339']
INFO:root:(127.0.0.1:55395) Agregating models.
INFO:root:(127.0.0.1:40133) Model added (4/4) from ['127.0.0.1:37107']
INFO:root:(127.0.0.1:40133) Agregating models.
INFO:root:(127.0.0.1:37107) Model added (3/4) from ['127.0.0.1:55395', '127.0.0.1:40133']
DEBUG:root:(127.0.0.1:37107) Can't add a model tha

DEBUG:root:Closed connection: 127.0.0.1:40133
DEBUG:root:(127.0.0.1:55395) (NodeConnection Loop) Exception: [Errno 104] Connection reset by peer
DEBUG:root:Closed connection: 127.0.0.1:55395
INFO:root:(127.0.0.1:40133) Evaluating...
INFO:root:(127.0.0.1:40133) Training...
INFO:root:(127.0.0.1:55395) Evaluating...
INFO:root:(127.0.0.1:40133) Model added (1/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:55395) Training...
INFO:root:(127.0.0.1:40133) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (1/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:40133) Model added (2/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:37339) Votes: {'127.0.0.1:55395', '127.0.0.1:40133', '127.0.0.1:37107'} of {'127.0.0.1:37339', '127.0.0.1:55395', '127.0.0.1:40133', '127.0.0.1:37107'}
INFO:root:(127.0.0.1:37107) Votes: {'127.0.0.1:37339', '

INFO:root:(127.0.0.1:37339) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:37107) Gossiping model to 3 train set nodes.
ERROR:root:(127.0.0.1:55395) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:55395) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:40133) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:40133) Tried to add a model while learning is not running
INFO:root:(127.0.0.1:37339) Round 2 of 2 finished.
INFO:root:(127.0.0.1:37339) Evaluating...
INFO:root:(127.0.0.1:37339) Taining finished!!.
ERROR:root:(127.0.0.1:37339) Tried to add a model while learning is not running
INFO:root:(127.0.0.1:37107) Round 2 of 2 finished.
INFO:root:(127.0.0.1:37107) Evaluating...
INFO:root:(127.0.0.1:37107) Taining finished!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 37107 y desconectándose de 3 nodos
DEBUG:root:Closed connection: 127.0.0.1:37107
DEBUG:root:Closed connection: 127.0.0.1:37107

In [35]:
from collections import defaultdict
import os
import json
from torch.utils.data import Dataset
from pytorch_lightning import LightningDataModule
import numpy
import torch
from torch.utils.data import DataLoader, Subset, random_split

class JSON_MNISTDataModule(LightningDataModule):
    def __init__(self, train_dataset, test_dataset, batch_size=32, num_workers=4, val_percent=0.1):
        super().__init__()
        train_samples = round(len(train_dataset)*(1-val_percent))
        self.train_dataset, self.val_dataset = random_split(train_dataset, [train_samples, len(train_dataset) - train_samples])
        self.test_dataset = test_dataset
        
        self.batch_size = batch_size
        self.num_workers = num_workers

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
    
class JSON_MNIST(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, samples, labels):
        """
        Args:
            samples (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.samples = torch.tensor(samples) 
        self.samples = torch.reshape(self.samples, (-1, 1, 28, 28)) # channels, width, height

        self.labels = torch.tensor(labels)
        assert len(self.samples)==len(self.labels)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx): 
        # adding a channel (b/w) 
        return self.samples[idx],self.labels[idx]
        

    
def get_samples(data_dir):
    train_data_dir = os.path.join(data_dir, 'data', 'train')
    test_data_dir = os.path.join(data_dir, 'data', 'test')

    # recordar cargarse groups
    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)
    
    result = []
    for u in users:
        return train_data[u]["x"]
    

def build_datamodules(data_dir):
    train_data_dir = os.path.join(data_dir, 'data', 'train')
    test_data_dir = os.path.join(data_dir, 'data', 'test')

    # recordar cargarse groups
    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)
    
    result = []
    for u in users:
        result.append( 
            JSON_MNISTDataModule( 
                JSON_MNIST(train_data[u]["x"],train_data[u]["y"]),
                JSON_MNIST(test_data[u]["x"],test_data[u]["y"])
            )
        )
    
    return result
    
    
    
def setup_clients(data_dir, model=None, use_val_set=False):
    eval_set = 'test' if not use_val_set else 'val'
    train_data_dir = os.path.join(data_dir, 'data', 'train')
    test_data_dir = os.path.join(data_dir, 'data', eval_set)

    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)

    return users, groups, train_data, test_data


def read_data(train_data_dir, test_data_dir):
    '''parses data in given train and test data directories
    assumes:
    - the data in the input directories are .json files with 
        keys 'users' and 'user_data'
    - the set of train set users is the same as the set of test set users
    
    Return:
        clients: list of client ids
        groups: list of group ids; empty list if none found
        train_data: dictionary of train data
        test_data: dictionary of test data
    '''
    train_clients, train_groups, train_data = read_dir(train_data_dir)
    test_clients, test_groups, test_data = read_dir(test_data_dir)

    assert train_clients == test_clients
    assert train_groups == test_groups

    return train_clients, train_groups, train_data, test_data



def read_dir(data_dir):
    clients = []
    data = defaultdict(lambda : None)

    files = os.listdir(data_dir)
    files = [f for f in files if f.endswith('.json')]
    for f in files:
        file_path = os.path.join(data_dir,f)
        with open(file_path, 'r') as inf:
            cdata = json.load(inf)
        clients.extend(cdata['users'])

        data.update(cdata['user_data'])

    clients = list(sorted(data.keys()))
    return clients, [], data



In [36]:
from pytorch_lightning import Trainer, LightningModule
import torch
from torch import nn
from torch.nn import functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy

class MLP(LightningModule):
    """
    Multilayer Perceptron (MLP) to solve MNIST with PyTorch Lightning.
    """

    def __init__(self, metric = Accuracy, lr_rate=0.001): # low lr to avoid overfitting
        
        # Set seed for reproducibility iniciialization
        seed = 666
        torch.manual_seed(seed)

        super().__init__()
        self.lr_rate = lr_rate
        self.metric = metric()

        # 10 clases
        self.l1 = torch.nn.Linear(28 * 28, 128)
        self.l2 = torch.nn.Linear(128, 256)
        self.l3 = torch.nn.Linear(256, 62)

    def forward(self, x):
        """
        """
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        x = torch.log_softmax(x, dim=1)
        return x

    def configure_optimizers(self):
        """
        """
        return torch.optim.Adam(self.parameters(), lr=self.lr_rate)

    def training_step(self, batch, batch_id):
        """
        """
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("train_loss", loss, prog_bar=True)
        return loss 

    def validation_step(self, batch, batch_idx):
        """
        """
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(self(x), y)
        out = torch.argmax(logits, dim=1)
        metric = self.metric(out, y) 
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_metric", metric, prog_bar=True)    
        return loss

    def test_step(self, batch, batch_idx):
        """
        """
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(self(x), y)
        out = torch.argmax(logits, dim=1)
        metric = self.metric(out, y) 
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_metric", metric, prog_bar=True)
        return loss
        

datamodules = build_datamodules("/home/pedro/Downloads/leaf/data/femnist")
tr = Trainer(max_epochs=10, accelerator="auto", logger=None, enable_checkpointing=False, enable_model_summary=False) 
tr.fit(MLP(), datamodules[55])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [25]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM

for batch in iter(MnistFederatedDM().train_dataloader()):
    print(batch[0].size())
    break

for batch in iter(datamodules[55].train_dataloader()):
    print(batch[0].size())
    break


Train: 54000 Val:6000 Test:10000
torch.Size([32, 1, 28, 28])
torch.Size([32, 1, 28, 28])


In [16]:

sample = get_samples("/home/pedro/Downloads/leaf/data/femnist")
sample = torch.tensor(sample)

In [21]:
sample.size()
sample = torch.reshape(sample, (-1, 1, 28,28))
sample.size()


torch.Size([256, 1, 28, 28])

In [118]:
torch.tensor([[1]])[0]

tensor([1])